外場は与えず、原子間斥力のみ与えて Quench して原子配置を最適化する。


In [5]:
from logging import getLogger

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

logger = getLogger()

# 斥力で配置を調整する原子の個数。最終的に配置する炭素の数ではない。
Natom = 192
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 初期配置。セル相対。
r = random_box(Natom)

# ポテンシャルエネルギーは粒子位置の関数。
pot = lambda r, cell: repulsive_potential(r, cell, repul=4, rc=2.0)
# その勾配。
dpot = lambda r, cell: -repulsive_force(r, cell, repul=4, rc=2.0)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 4406.270264
         Iterations: 16
         Function evaluations: 264
         Gradient evaluations: 251


/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-z6wFDqgC-py3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1659: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



Quench 前とあとを比較。


In [6]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go
from itertools import starmap

x = r @ cell

# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:, 0],
            y=x[:, 1],
            z=x[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

外場を加えたエネルギー関数とその傾きの関数を準備する。


In [3]:
# The gyroid surface and its gradient
from logging import getLogger, basicConfig, INFO

import numpy as np

from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

from graphenestudio.surface import Gyroid, Sphere, Diamond, PSurface

# from graphenestudio.surface.psurface import PSurface


basicConfig(level=INFO)
logger = getLogger()

# 斥力粒子の個数
Natom = 192 * 10
# 外場の強度。小さいと、ぎゅうぎゅうづめになった時に面外に粒子がこぼれる可能性がある。
cost = 100000
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 曲面関数
surf = Gyroid(eccentricity=0.0)
# surf = Sphere(radius=0.45)
# surf = Diamond(eccentricity=0.0)
# surf = PSurface(eccentricity=0.0)

# 斥力粒子の座標(セル相対)
r = random_box(Natom)

# 斥力粒子のポテンシャルエネルギー関数
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
# その勾配
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

Quench し、表示する。


In [4]:
from graphenestudio.quench import quench_particles
import plotly.graph_objects as go

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

         Current function value: 11590793.157681
         Iterations: 55
         Function evaluations: 578
         Gradient evaluations: 556


/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-LdnOtBEG-py3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1658: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



Quench した構造に有限温度を与えてすこし原子が動けるようにし、100 ステップの疑似分子動力学法を実施する。これにより、より歪みが少ない形状にたどりつけるはず。

斥力粒子を均質に配置して三角格子をつくったら、三角格子の重心同士を連結して、双対グラフを生成する。これがグラフェンの構造となる。一連の処理は graphenate 関数の内部で行われる。

ここでは結果を yaplot 形式で出力している。


In [14]:
from graphenestudio import draw_yaplot, graphenate, dump_gro


base = "psurface"
with open(f"{base}.gro", "w") as gro:
    with open(f"{base}.yap", "w") as file:
        count = 100
        for x, cell, g in graphenate(
            Natom,
            cell,
            pot,
            dpot,
            dt=0.0005,  # 0.005
            T=0.1,
        ):
            print(count)
            file.write(draw_yaplot(x, cell, g))
            dump_gro(x, cell, g, gro)

            count -= 1
            if count == 0:
                break

/Users/matto/Library/Caches/pypoetry/virtualenvs/graphene-studio-z6wFDqgC-py3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1659: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



         Current function value: 22884.856550
         Iterations: 76
         Function evaluations: 230
         Gradient evaluations: 225


INFO:root:[240, 35, 14, 10] 3-6 cycles in the packing
INFO:root:Tempering
INFO:root:[240, 35, 14, 10] 3-6 cycles in the packing
INFO:root:[254, 31, 17, 7] 3-6 cycles in the packing
INFO:root:[270, 25, 18, 6] 3-6 cycles in the packing
INFO:root:[280, 25, 15, 5] 3-6 cycles in the packing
INFO:root:[280, 28, 13, 5] 3-6 cycles in the packing
INFO:root:[280, 28, 13, 5] 3-6 cycles in the packing
INFO:root:[283, 28, 12, 5] 3-6 cycles in the packing
INFO:root:[289, 25, 11, 5] 3-6 cycles in the packing
INFO:root:[295, 22, 11, 5] 3-6 cycles in the packing
INFO:root:[301, 23, 11, 3] 3-6 cycles in the packing
INFO:root:[303, 25, 9, 3] 3-6 cycles in the packing
INFO:root:[315, 22, 7, 3] 3-6 cycles in the packing
INFO:root:[319, 22, 7, 2] 3-6 cycles in the packing
INFO:root:[320, 22, 8, 1] 3-6 cycles in the packing
INFO:root:[326, 21, 8, 0] 3-6 cycles in the packing
INFO:root:[330, 19, 8, 0] 3-6 cycles in the packing
INFO:root:[340, 14, 8, 0] 3-6 cycles in the packing
INFO:root:[343, 14, 7, 0] 3-6 c

100


INFO:root:[390, 1, 0, 0] 3-6 cycles in the packing


...

3


INFO:root:[380, 6, 0, 0] 3-6 cycles in the packing
INFO:root:[0, 0, 2, 164, 26, 0] 3-8 cycles in the graph


2


INFO:root:[380, 6, 0, 0] 3-6 cycles in the packing
INFO:root:[0, 0, 3, 162, 27, 0] 3-8 cycles in the graph


1


曲面を解析的な式ではなく、グリッド上の数値配列として与える。

`cube`や`sphere`は 3 次元グリッドで定義される$w=f(x,y,z)$型の空間関数で、これが$f(x,y,z)=0$を横切る isosurface で、graphene の形状を指定する。


In [13]:
from logging import INFO, basicConfig, getLogger, DEBUG

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force
from graphenestudio.surface import Grid, Ticks, GridSurfaceFunc


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))

r = 0.4  # "radius" of the cube, i.e., half length of an edge

XYZ = np.mgrid[-0.5:0.5:11j, -0.5:0.5:11j, -0.5:0.5:11j]
xticks = Ticks(min=-0.5, binw=0.1)
yticks = Ticks(min=-0.5, binw=0.1)
zticks = Ticks(min=-0.5, binw=0.1)

# グリッドタイプの関数定義法
cube = Grid(
    values=np.max(np.abs(XYZ), axis=0) - r, xticks=xticks, yticks=yticks, zticks=zticks
)

sphere = Grid(
    values=np.sum(XYZ**2, axis=0) - r * r,
    xticks=xticks,
    yticks=yticks,
    zticks=zticks,
)

surf = GridSurfaceFunc(sphere)

In [14]:
r = random_box(Natom)
# x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 19660.800824
         Iterations: 15
         Function evaluations: 215
         Gradient evaluations: 203


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



In [15]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()